In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats 
import numpy as np 
import statsmodels.api as sm 
from sklearn.cluster import KMeans 
import scikit_posthocs as sp 
from scipy.stats import kruskal 
from sklearn.utils import resample 
from sklearn.feature_selection import RFE 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score 
 
pd.set_option('display.max_columns', None)  # Отображение всех столбцов 
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк    
regions = pd.read_csv('regions.csv') 
starts = pd.read_csv('starts.csv') 
volunteers = pd.read_csv('volunteers.csv') 
 
 
 
volunteers = volunteers[volunteers.athlete_age < 100] 
starts = starts[starts.athlete_age < 80] 
starts = starts[starts.finish_time < 5000] 
 
starts['rain.1h'] = starts['rain.1h'].fillna(0) 
starts['snow.1h'] = starts['snow.1h'].fillna(0) 
 
cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg( 
    cnt_runners = ('athlete_id', 'count') 
) 
 
 
 
 
ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique() 
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique() 
common_athletes = set(ages_14) & set(ages_15) 
starts = starts[~starts['athlete_id'].isin(common_athletes)] 
starts = starts[starts['athlete_age'] > 14] 
 
 
starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0) 
 
 
volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)   
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')   
 
starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было 
 
print(len(starts)) 
unique_dates = ( 
    pd.concat([ 
        starts[['athlete_id', 'event_date']], 
        volunteers[['athlete_id', 'event_date']] 
    ]) 
    .drop_duplicates() 
) 
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count') 
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int) 
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left') 
 
 
# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]   
starts = starts[starts.event_date <= '2023-11-15'] 
 
 
 
newbie = starts[starts['is_newbie'] == 1] 
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id') 
didnt_come = first_runs[first_runs.is_repeated == 0] 
 
come = first_runs[first_runs.is_repeated == 1] 
come_with_didnt_come = pd.concat([come, didnt_come]) 
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1) 
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left") 
print(len(come_with_didnt_come))


166516
18114
